In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime as dt
import openpyxl

from masakali_data.utils import open_csv, save_csv, current_directory, open_parquet
from masakali_data.exchange_rates import convert_columns_to_usd, convert_columns_to_idr
# from masakali_data.airbnb import calculate_payouts


current_directory = current_directory()
current_year = dt.now().year


In [3]:
df = open_parquet(current_directory, 'data/airbnb_revenue_idr.parquet')

,booking_id,platform,villa_name,guest_name,check_in,check_out,nights,total_payout,property_use_tax,host_service_fee,vat,net_income,cleaning_fee,taxable_income
0,HMA8BMS38Q,airbnb,Chandra: Entire villa w/ private pool and kitchen,Daniel Jaramillo,2022-12-28,2022-12-31,3,4335000.00,0.0,765000.0,0.00,5100000.0,0.0,5100000.0
1,HMKK83P4FK,airbnb,Chandra: Entire villa w/ private pool and kitchen,Melanie Weiss,2022-12-25,2022-12-28,3,4335000.00,0.0,765000.0,0.00,5100000.0,0.0,5100000.0
2,HMFN9WTYHD,airbnb,Chandra: Entire villa w/ private pool and kitchen,Laura Seire,2022-12-24,2022-12-25,1,1750271.35,0.0,315000.0,34728.65,2100000.0,400000.0,1700000.0
3,HMTWMP9M3Y,airbnb,Surya: Luxury Villa with kitchen and private pool,Khrystyna Kozlova,2022-12-23,2022-12-27,4,5780000.00,0.0,1020000.0,0.00,6800000.0,0.0,6800000.0
4,HM2ZWTQHYA,airbnb,Surya: Luxury Villa with kitchen and private pool,Денис Харламов,2022-12-21,2022-12-22,1,1537865.86,0.0,276750.0,30384.14,1845000.0,400000.0,1445000.0
5,HMPWMNWBTQ,airbnb,Chandra: Entire villa w/ private pool and kitchen,Gunjan Shah,2022-12-20,2022-12-24,4,5780000.00,0.0,1020000.0,0.00,6800000.0,0.0,6800000.0
6,HMYNJT4ZFH,airbnb,Surya: Luxury Villa with kitchen and private pool,Promita Daniel,2022-12-19,2022-12-20,1,1445000.00,0.0,255000.0,0.00,1700000.0,0.0,1700000.0
7,HM9SENE9YX,airbnb,Chandra: Entire villa w/ private pool and kitchen,Katie Coates,2022-12-17,2022-12-20,3,4335000.00,0.0,765000.0,0.00,5100000.0,0.0,5100000.0
8,HMPBSHQYPX,airbnb,Surya: Luxury Villa with kitchen and private pool,Aakanksha Aakanksha,2022-12-17,2022-12-19,2,2890000.00,0.0,510000.0,0.00,3400000.0,0.0,3400000.0
9,HMA8JKWEDY,airbnb,Surya: Luxury Villa with kitchen and private pool,Elena Merget,2022-12-16,2022-12-17,1,1870000.00,0.0,330000.0,0.00,2200000.0,400000.0,1800000.0


In [12]:


def calculate_check_out(df):
    df['check_in'] = pd.to_datetime(df['check_in'])
    df['check_out'] = df['check_in'] + pd.to_timedelta(df['nights'], unit='d')
    return df

def format_date(df, column):
    df[column] = df[column].dt.strftime('%Y-%m-%d')
    return df

def create_monthly_sheets(df, filename):    
    with pd.ExcelWriter(filename) as writer:
        for month in df['check_out'].dt.month.unique():
            month_df = df[df['check_out'].dt.month == month].copy()
            month_name = month_df['check_out'].dt.month_name().iloc[0]
            month_df = format_date(month_df, 'check_in')
            month_df = format_date(month_df, 'check_out')
            month_df.to_excel(writer, sheet_name=month_name, index=False)

        df['check_out'] = pd.to_datetime(df['check_out']).dt.to_period('M').dt.to_timestamp()
        totals_df = df.groupby(pd.Grouper(key='check_out', freq='M')).sum(numeric_only=True).reset_index()
        totals_df = format_date(totals_df, 'check_out')
        totals_df.rename({'check_out': 'month'}, axis=1, inplace=True)
        totals_df.to_excel(writer, sheet_name='Totals', index=False)


def create_sheet_for_bre(df):
    df['cost_less_property_tax'] = df['cost_to_guest'] - df['property_use_tax']
    df['check_out'] = pd.to_datetime(df['check_out']).dt.to_period('M').dt.to_timestamp()

    df = df[['date', 'check_out', 'cost_less_property_tax', 'nightly_adjustment', 'host_service_fee']]
    df_usd = convert_columns_to_usd(df.copy(), ['cost_less_property_tax', 'nightly_adjustment', 'host_service_fee'])

    totals_idr = df.groupby(pd.Grouper(key='check_out', freq='M')).sum(numeric_only=True).reset_index()
    totals_idr = format_date(totals_idr, 'check_out')
    totals_idr.rename({'check_out': 'month'}, axis=1, inplace=True)
    save_csv(current_directory, totals_idr, f'airbnb_totals_2023_idr.csv')

    totals_usd = df_usd.groupby(pd.Grouper(key='check_out', freq='M')).sum(numeric_only=True).reset_index()
    totals_usd = format_date(totals_usd, 'check_out')
    totals_usd.rename({'check_out': 'month'}, axis=1, inplace=True)
    save_csv(current_directory, totals_usd, f'airbnb_totals_2023_usd.csv')


def calculate_payouts(payouts, year=current_year):
    df = payouts[["Date", "Paid Out", "Currency"]].copy()
    df.rename(columns={"Date": "date", "Paid Out": "USD"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
    df["IDR"] = df["USD"]
    df = convert_columns_to_idr(df, ["IDR"])
    df = df[["date", "IDR", "USD"]]
    save_csv(current_directory, df, f'{year}/payouts_{year}.csv')
    

def parse_month_data(year=current_year):
    df = open_csv(current_directory, f'{year}/all_{year}.csv')
    nightly_rate_adjustments = open_csv(current_directory, f'nightly_rate_adjustments.csv')
    nightly_rate_adjustments = nightly_rate_adjustments.astype({'booking_id': str, 'nightly_adjustment': float})
    ## Payouts
    payout = df[(df['Type'] == 'Payout')]
    payout = calculate_payouts(payout, year)

    ## Taxes
    use_tax = df[df['Type'] == 'Pass Through Tot'][['Confirmation Code', 'Amount']]
    use_tax = use_tax.rename(columns={'Amount': 'property_use_tax'})

    ## Resolution adjustment
    resolutions = df[(df['Type'].isin(['Resolution Adjustment', 'Resolution Payout']))][['Confirmation Code', 'Amount']]
    resolutions = resolutions.rename(columns={'Amount': 'resolution'})

    ## Reservations
    reservations = df[df['Type'] == 'Reservation']
    
    ## Grab the relevant columns
    reservations = reservations[['Date', 'Confirmation Code', 'Start Date', 'Nights', 'Guest', 'Amount', 'Host Fee', 'Cleaning Fee', 'Listing']]
    

    reservations = reservations.merge(use_tax, how='left', on='Confirmation Code')
    reservations['property_use_tax'] = reservations['property_use_tax'].fillna(0)   
    reservations = reservations.merge(resolutions, how='left', on='Confirmation Code')
    reservations['resolution'] = reservations['resolution'].fillna(0)



    ## Rename columns
    reservations = reservations.rename(columns={
        'Date': 'date',
        'Confirmation Code': 'booking_id', 
        'Start Date': 'check_in',
        'Nights': 'nights', 
        'Guest': 'guest_name', 
        'Amount': 'total', 
        'Host Fee': 'commission', 
        'Cleaning Fee': 'cleaning_fee',
        'Currency': 'currency',
        'Listing': 'villa_name',
    })
    
    reservations = calculate_check_out(reservations)
    reservations = reservations.merge(nightly_rate_adjustments, how='left', on='booking_id')
    reservations = reservations.fillna(0)

    

    reservations['platform'] = 'airbnb'
    reservations['host_service_fee'] =(reservations.total + reservations.commission) * 0.15
    reservations['vat'] = reservations.commission - reservations.host_service_fee
    reservations['taxable_income'] = reservations.total - reservations.cleaning_fee - reservations.property_use_tax
    reservations['total_payout'] = reservations.total + reservations.property_use_tax
    reservations['cost_to_guest'] = reservations.total_payout + reservations.commission
    # reservations['cost_for_room'] = reservations.cost_to_guest - reservations.property_use_tax - reservations.cleaning_fee
    
    print(reservations.head())
    # Add columns
    # reservations['platform'] = 'airbnb'
    # reservations['total_payout'] = reservations.gross_income + reservations.property_use_tax
    # reservations['host_service_fee'] =(reservations.gross_income + reservations.commission) * 0.15
    # reservations['vat'] = reservations.commission - reservations.host_service_fee
    # reservations['net_income'] = reservations.total_payout - reservations.property_use_tax + reservations.host_service_fee + reservations.vat
    # reservations['taxable_income'] = reservations.net_income - reservations.cleaning_fee

    columns_to_convert = ['total_payout','nightly_adjustment', 'property_use_tax', 'host_service_fee', 'vat', 'cost_to_guest', 'cleaning_fee', 'taxable_income']
    


    reservations = reservations[['date', 'platform', 'booking_id', 'villa_name', 'guest_name', 'check_in', 'check_out', 'cost_to_guest', 'cleaning_fee', 'nightly_adjustment', 'property_use_tax', 'host_service_fee','vat', 'total_payout','taxable_income']]
    reservations = reservations.round(2)

    create_sheet_for_bre(reservations)

    df_usd = convert_columns_to_usd(reservations.copy(), columns_to_convert)
    
    reservations.drop(columns=['date'], inplace=True)
    df_usd.drop(columns=['date'], inplace=True)
    

    save_csv(current_directory, reservations, f'{year}/airbnb_revenue_idr_{year}.csv')
    create_monthly_sheets(reservations, f'{year}/airbnb_revenue_idr_{year}.xlsx')


    
    save_csv(current_directory, df_usd, f'{year}/airbnb_revenue_usd_{year}.csv')
    create_monthly_sheets(df_usd, f'{year}/airbnb_revenue_usd_{year}.xlsx')

    return df_usd
    

df = parse_month_data(2023)


# create_monthly_sheets(df, 'airbnb_revenue_2023.xlsx')

         date  booking_id   check_in  nights       guest_name      total  \
0  12/07/2023  HM3HSSWMF8 2023-12-06     3.0     Nadia Brault  4787233.0   
1  12/05/2023  HMNABKWC3J 2023-12-04     2.0     Neil Whybrow  2934219.0   
2  11/29/2023  HMXZERCB95 2023-11-28     1.0      Edah Abueva  1527629.0   
3  11/27/2023  HM295ZBZ93 2023-11-26     1.0  Mikko Wallenius  1349484.0   
4  11/17/2023  HM4TFEQXMY 2023-11-16     1.0    Fede Franzosi  2040000.0   

   commission  cleaning_fee  \
0    844806.0      400000.0   
1    517803.0      400000.0   
2    269581.0      400000.0   
3    238144.0      400000.0   
4    360000.0      400000.0   

                                          villa_name  property_use_tax  \
0  Chandra: Entire villa w/ private pool and kitchen         1182785.0   
1  Chandra: Entire villa w/ private pool and kitchen          724889.0   
2  Chandra: Entire villa w/ private pool and kitchen          377415.0   
3  Surya: Luxury Villa with kitchen and private pool        